In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import os
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async" # this seems to be highly important for totally utilizing your GPU's memory, but it also breaks the profiler's memory breakdown
# note that GradientTape needs several times the memory needed to compute the fidelity of a single circuit

import numpy as np
import qutip as qt
from QOGS.optimizer.tf_adam_optimizer import AdamOptimizer
from QOGS.gate_sets import trajGRAPE, GRAPE
import matplotlib.pyplot as plt


Need tf version 2.3.0 or later. Using tensorflow version: 2.8.0



In [2]:
q_dim = 3
c_dim = 10
DAC_time_resolution = 2 # in ns

# In GHz = cycles / ns
anharm = -.180
kerr = -1e-5*0
chi = -0.25e-3
chi_prime = 0
drive = D = 2 * np.pi * 1e-3
# jump_op = qt.tensor(qt.qeye(c_dim), qt.fock_dm(q_dim, 0)) / 2 - qt.tensor(qt.qeye(c_dim), qt.fock_dm(q_dim, 2)) / 2
jump_op = qt.tensor(qt.qeye(c_dim), qt.fock_dm(q_dim, 0))

a = qt.tensor(qt.destroy(c_dim), qt.qeye(q_dim))
b = qt.tensor(qt.qeye(c_dim), qt.destroy(q_dim))
gf = qt.tensor(qt.qeye(c_dim), qt.basis(q_dim, 0) * qt.basis(q_dim, 2).dag())
fg = gf.dag()
ad = a.dag()
bd = b.dag()
H0 = (chi) * ad * a * bd * b
H0 += (kerr/2) * ad * ad * a * a
H0 += (chi_prime / 6) * ad * ad * a * a * bd * b
H0 -= 1j / 2 * jump_op.dag() * jump_op * 1 / 5500
H0 *= 2*np.pi
Hcs = [D*(gf + fg), 1j*D*(gf - fg)]

init_states = [
    ((qt.tensor(qt.basis(c_dim, 0), qt.basis(q_dim, 0)) 
        + qt.tensor(qt.basis(c_dim, 4), qt.basis(q_dim, 0))) / np.sqrt(2)
        + qt.tensor(qt.basis(c_dim, 2), qt.basis(q_dim, 0))).unit()
]

final_states = [
    ((qt.tensor(qt.basis(c_dim, 0), qt.basis(q_dim, 2)) 
        + qt.tensor(qt.basis(c_dim, 4), qt.basis(q_dim, 2))) / np.sqrt(2)
        - qt.tensor(qt.basis(c_dim, 2), qt.basis(q_dim, 2))).unit()
]

success_op = qt.tensor(qt.qeye(c_dim), qt.fock_dm(q_dim, 2))


In [3]:
synth_params = {
    'N_blocks': 601, # note that the length of the pulse is this times the DAC_time_resolution
    'N_multistart' : 3, #Batch size (number of circuit optimizations to run in parallel)
    'epochs' : 3000, #number of epochs before termination
    'epoch_size' : 5, #number of adam steps per epoch
    'learning_rate' : 0.1, #adam learning rate
    'term_fid' : 0.999, #0.995, #terminal fidelitiy
    'dfid_stop' : -1, #stop if dfid between two epochs is smaller than this number
    'initial_states' : init_states, #qubit tensor oscillator, start in |g> |0>
    'target_states' : final_states, #end in |e> |target>.
    'name' : 'trajGRAPE SNAP binom Z', #name for printing and saving
    'use_phase' : True,
    'filename' : None, #if no filename specified, results will be saved in this folder under 'name.h5'
}

# We initialize the gateset here
gate_set_params = {
    'H_static' : H0,
    'H_control' : Hcs,
    'DAC_delta_t' : DAC_time_resolution,
    'inplace' : False, # true uses less memory, but is slower. Just use false
    'scale' : 1.0, # range of DAC amplitudes for initial random waves
    'bandwidth' : 0.1,
    'ringup' : 40,
    'jump_ops' : jump_op,
    'success_op' : success_op,
    'threshold_start' : 0.7,
    'threshold_end' : 0.75,
    'n_traj' : 1000,
    'gatesynthargs': synth_params
}



GRAPE_gate_set = trajGRAPE(**gate_set_params)

2022-07-25 14:11:42.024994: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-25 14:11:42.653025: I tensorflow/core/common_runtime/gpu/gpu_process_state.cc:214] Using CUDA malloc Async allocator for GPU: 0
2022-07-25 14:11:42.653206: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30990 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:86:00.0, compute capability: 7.0


In [4]:
#create optimization object. 
#initial params will be randomized upon creation
opt = AdamOptimizer(GRAPE_gate_set)

#print optimization info. this lives in gatesynth, since we eventually want to fully abstract away the optimizer
GRAPE_gate_set.best_fidelity()

2022-07-25 14:11:44.691222: I tensorflow/core/util/cuda_solvers.cc:179] Creating GpuSolver handles for stream 0x55631de01db0


4.62735e-05

In [ ]:
#run optimizer.
#note the optimizer can be stopped at any time by interrupting the python consle,
#and the optimization results will still be saved and part of the opt object.
#This allows you to stop the optimization whenever you want and still use the result.
# Note that you will not want to use the performance profiler while using 'inplace' mode. You will run out of memory
opt.optimize()#logdir='logs')

Start time: 2022-07-25 14:11:45
 Epoch: 838 / 3000 Max Fid: 0.595456 Avg Fid: 0.590816 Max dFid: 0.000398 Avg dFid: -0.005385 Elapsed time: 1:12:45.387727 Expected remaining time: 3:07:42.4919646

In [ ]:
GRAPE_gate_set.print_info()

In [ ]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [ ]:
# Launch TensorBoard and navigate to the Profile tab to view performance profile
%tensorboard --logdir=logs